<a href="https://colab.research.google.com/github/cjfghk5697/Classificiation_BMD/blob/main/eff_7(test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install timm torchmetrics

     |████████████████████████████████| 376 kB 7.2 MB/s 
     |████████████████████████████████| 332 kB 70.5 MB/s 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.optim as optim
import torch.nn as nn
import timm
import datetime
import csv

from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchmetrics import AUROC, ROC
from pathlib import Path

import gdown
global PATH
import os
import time
import datetime
import logging
import random
import torch
from torch.autograd import Variable

from google.colab import files
from torch.optim.lr_scheduler import _LRScheduler
from torchsummary import summary
from pathlib import Path
from PIL import Image

import torchvision.models as models

from torch.utils.data.dataset import random_split

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F
from tqdm import tqdm
import argparse
import albumentations as A
import cv2
from glob import glob
from albumentations.core.transforms_interface import ImageOnlyTransform
from matplotlib import pyplot as plt
import math
import timm
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dir = '/content/drive/MyDrive/bmd_dataset/train'
model_root = './model/'

In [5]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

# Train

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Model list

In [7]:
# all_densenet_models = timm.list_models('**')
# all_densenet_models

Model

In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('efficientnet_b7', pretrained=False)
        self.backbone.classifier = nn.Identity()
        self.fc = nn.Linear(1000, 3)

    def forward(self, x):
        return self.fc(self.backbone(x))

Making a dataset

In [12]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = ImageFolder(train_dir, transform=train_transform)

# normal: 0, osteopenia:1, osteoporosis:2

Train & Validation

In [15]:
net = Model()
net.train()
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = RAdam(net.parameters(), lr=1e-3, betas=(0.9, 0.999), weight_decay=1e-4)

In [16]:
def save_model(model, acc, date_time, name):
    model_name = name + '.pth'
    model_path = Path(model_root + date_time)
    model_path.mkdir(parents=True, exist_ok=True)
    print('Saving model (Accuracy {:.2f}%) to {}'.format(acc*100, str(model_path / model_name)) )
    torch.save({'model_state_dict':model.state_dict(), 'acc':acc}, str(model_path / model_name) )

In [17]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, auc=False, device=device):
    tz = datetime.timezone(datetime.timedelta(hours=9)) # Timezone infomation
    date_time = datetime.datetime.now(tz).strftime('%Y-%m-%d-%H-%M-%S')
    
    best_val_acc = 0
    best_train_acc = 0
    epoch_train_acc = 0
    
    for epoch in range(num_epochs + 1):
        print('\n------------------------')
        print('EPOCH {}/{}'.format(epoch, num_epochs))
        print('------------------------')
        
        if auc == True:
            auc_roc_metric = AUROC(num_classes=3, average=None)
            roc_metric = ROC(num_classes=3)

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            total_loss = 0
            total_size = 0
            total_corrects = 0
            
            # 학습 전 성능 확인
            if (epoch == 0) and (phase == 'train'):
                continue
            
            num_iteration = len(dataloaders_dict[phase])
            
            for idx, (inputs, labels) in enumerate(tqdm(dataloaders_dict[phase])):
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss = loss.item()
                    total_loss += epoch_loss

                    epoch_corrects = torch.sum(preds == labels.data)
                    epoch_acc = epoch_corrects.double() / inputs.size(0)
                    total_corrects += epoch_corrects
                    total_size += inputs.size(0)
                    
                    if phase == 'train':
                        print('{} [{}/{}] LOSS: {:.4f} ACC: {:.4f}'.format(phase, idx+1, num_iteration, epoch_loss, epoch_acc))

                    if (phase == 'val') & (auc == True):
                        auc_roc_metric(outputs, labels)
                        roc_metric(outputs, labels)
            
            epoch_loss_avg = total_loss / num_iteration
            epoch_acc_avg = total_corrects / total_size
            
            if phase == 'train':
                if best_train_acc < epoch_acc_avg:
                    best_train_acc = epoch_acc_avg
                print('{} LOSS: {:.4f} ACC: {:.4f} BEST ACC: {:.4f}'.format(phase, epoch_loss_avg, epoch_acc_avg, best_train_acc))

            if phase == 'val':
                if (epoch_acc_avg == best_val_acc) & (epoch_acc_avg == best_train_acc):
                    save_model(net, epoch_acc_avg, date_time, 'best')
                elif best_val_acc < epoch_acc_avg:
                    best_val_acc = epoch_acc_avg
                    print('Best Validation Accuracy: {:.4f}'.format(epoch_acc_avg))
                    save_model(net, epoch_acc_avg, date_time, 'best')

                print('{} LOSS: {:.4f} ACC: {:.4f} BEST ACC: {:.4f}\n'.format(phase, epoch_loss_avg, epoch_acc_avg, best_val_acc))

                if auc == True:
                    for auroc, fpt, tpr, thresholds in zip(auc_roc_metric.compute(), *roc_metric.compute()):
                        print('auc: {:5.2f}'.format(auroc* 100))
                        size = min(len(fpt), len(tpr))
                        plt.plot(fpt[:size].cpu(), tpr[:size].cpu())
                        plt.show()
                        plt.cla()

In [18]:
num_epochs=50

batch_size = 32
num_workers = 4
view_auc = False

train_size = int(0.7*len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=num_workers)

dataloaders_dict = {"train": train_loader, "val": val_loader}

train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, view_auc)


------------------------
EPOCH 0/50
------------------------


  0%|          | 0/29 [00:20<?, ?it/s]


RuntimeError: ignored